In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv("passwords1.txt", header=None, delim_whitespace=True)
df.head()

,0
0,JV#0$S;D@j+c96/8C(W%
1,"J,)e+LH'ENmdV5YGWu4B"
2,tR0V2@$SNuU9;<&DafxO
3,/DXhI;GNl*nbCzKso=?@
4,b*ZB/egd=!XcvV>K(qY@


In [3]:
len(df)

95234682

In [4]:
df = df.rename(columns={0: "passwords"})
df.head()

,passwords
0,JV#0$S;D@j+c96/8C(W%
1,"J,)e+LH'ENmdV5YGWu4B"
2,tR0V2@$SNuU9;<&DafxO
3,/DXhI;GNl*nbCzKso=?@
4,b*ZB/egd=!XcvV>K(qY@


In [7]:
df.to_csv('df.csv')

Now we create a dictionary where every password has 20 values that are the ascii values for every character.

In [12]:
dic_ascii = {}
for i in df['passwords']:
    if i not in  dic_ascii:
        asc = []
        for c in i:
            asc.append(ord(c))
        dic_ascii[i] = asc
with open('dic_ascii.json', 'w') as fp:
    json.dump(dic_ascii, fp)

In [1]:
A = [1,2]

In [8]:
pip install sympy

Note: you may need to restart the kernel to use updated packages.


I choose m in respect of a 0.07% false positive rate that I will pose in the furture bloom filter, so the module of the hash fucntions will the the first greater prime of 527114570.

In [14]:
import sympy as s
m = s.nextprime(527114570)
m

527114611

In [16]:
import random
random.randrange(m)

298894430

I will create 4 hash functions, that I will later describe, the first thing is that I need 4 vectors with 20 components, all randomly generated in between 0 and m.

In [21]:
def randomizr(m): 
    vec = []
    for i in range(20):
        vec.append(random.randrange(m))
    return vec


In [23]:
vector1 = randomizr(m)
vector2 = randomizr(m)
vector3 = randomizr(m)
vector4 = randomizr(m)
print(vector1,'\n',vector2, '\n', vector3, '\n', vector4)
print(vector1 == vector2 or vector1==vector3 or vector1==vector4 or vector2 == vector3 or vector3==vector4)

[55562445, 326412123, 517620619, 82557444, 124484253, 206132606, 152363464, 457642380, 110539775, 141714105, 98519460, 169495199, 205943578, 356272723, 197821188, 525494969, 332311367, 172269659, 21190289, 261606644] 
 [146939973, 472828821, 362051872, 481260545, 150893091, 479208025, 161427083, 162709789, 333130560, 255061743, 408229312, 408864144, 110787110, 64801791, 305797461, 353665148, 379748443, 463555711, 301828971, 366943646] 
 [237904666, 8715804, 246860143, 36593556, 156812513, 93434722, 475668356, 319144638, 326069300, 113067605, 92330184, 427153890, 20137911, 163069086, 203737011, 76027686, 136271340, 262343691, 376044511, 161732585] 
 [21836976, 437493629, 93160995, 181676887, 443639206, 242915781, 419601293, 488777532, 481969058, 60534811, 26981764, 517617281, 482857327, 59545467, 445871635, 462668141, 373812211, 476308162, 15572568, 471346767]
False


Now that the vectors are all different, to avoid changing them every time we run the file, we save the vectors with the values printed above. We did that to make explicit the uniform distribution to chooes those components.

In [24]:
vector1 = [55562445, 326412123, 517620619, 82557444, 124484253, 206132606, 152363464, 457642380, 110539775, 141714105, 98519460, 169495199, 205943578, 356272723, 197821188, 525494969, 332311367, 172269659, 21190289, 261606644]
vector2 = [146939973, 472828821, 362051872, 481260545, 150893091, 479208025, 161427083, 162709789, 333130560, 255061743, 408229312, 408864144, 110787110, 64801791, 305797461, 353665148, 379748443, 463555711, 301828971, 366943646] 
vector3 = [237904666, 8715804, 246860143, 36593556, 156812513, 93434722, 475668356, 319144638, 326069300, 113067605, 92330184, 427153890, 20137911, 163069086, 203737011, 76027686, 136271340, 262343691, 376044511, 161732585]
vector4 = [21836976, 437493629, 93160995, 181676887, 443639206, 242915781, 419601293, 488777532, 481969058, 60534811, 26981764, 517617281, 482857327, 59545467, 445871635, 462668141, 373812211, 476308162, 15572568, 471346767]
#done!

Now we define the 4 hash fucntions, that will actually be the same but with different coefficients given by the vectors we just defined.